In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()


In [ ]:
# Summarise the data by UK Cardholder Household Domestic Spending All Quarterly --------------- Cardholders' Number Total Quarterly 

UK_cardholder_Dom_HH = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM' 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_Dom_All = bq.read_bq_table_sql(client, UK_cardholder_Dom_HH)
df_by_Dom_All.head()

# Caculate UK Domestic Total Spending Quarterly

# Assuming df_by_Dom_All is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_Dom_All is not None and has the expected columns
if df_by_Dom_All is not None and 'time_period_value' in df_by_Dom_All.columns and 'spend' in df_by_Dom_All.columns:
    # Group by quarter and sum the spend
    UK_cardholder_Dom_HH = df_by_Dom_All.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_cardholder_Dom_HH = UK_cardholder_Dom_HH.rename(columns={'cardholders': 'Dom_HH_All_cardholders'})
    print(UK_cardholder_Dom_HH)
else:
    print("DataFrame is empty or missing required columns.")
    
# Save the result to a CSV file
csv_filename = "UK_cardholder_Dom_HH.csv"
UK_cardholder_Dom_HH.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")


In [ ]:
# Summarise the data by UK Household Cardholder Abroad Spending All Quarterly --------------- Cardholders' Number Total Quarterly 

UK_cardholder_Intl_HH = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_Intl_HH = bq.read_bq_table_sql(client, UK_cardholder_Intl_HH)
df_by_Intl_HH.head()

# Caculate UK Abroad Total Spending Quarterly

# Assuming df_by_Intl_HH is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_Intl_HH is not None and has the expected columns
if df_by_Intl_HH is not None and 'time_period_value' in df_by_Intl_HH.columns and 'spend' in df_by_Intl_HH.columns:
    # Group by quarter and sum the spend
    UK_cardholder_Intl_HH = df_by_Intl_HH.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_cardholder_Intl_HH = UK_cardholder_Intl_HH.rename(columns={'cardholders': 'Intl_HH_All_cardholders'})
    print(UK_cardholder_Intl_HH)
else:
    print("DataFrame is empty or missing required columns.")
    
    # Save the result to a CSV file
csv_filename = "UK_cardholder_Intl_HH.csv"
UK_cardholder_Intl_HH.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")


In [ ]:
# Total UK Cardholder Household Domestic Quarterly

# Summarise the data by UK Cardholder Domestic Household Spending Total Quarterly

UK_spending_HH_Dom_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
and mcc = 'All'  
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and destination_country = 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_HH_Dom_All = bq.read_bq_table_sql(client, UK_spending_HH_Dom_All)
df_by_HH_Dom_All = df_by_HH_Dom_All.rename(columns={'total_spend': 'Dom_spend_HH'})
df_by_HH_Dom_All.head()


# Save the DataFrame to a CSV file
csv_filename = "UK_HH_Dom_Spending.csv"
df_by_HH_Dom_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Formula : Adjusted Quarterly Spend=( Dom_HH_All_cardholders (2019Q1) / Dom_HH_All_cardholders (quarter) ) × Dom_spend_All (quarter)

# UK Household Domestic Total Adjusted base value 2019Q1  ---------------- MODIFIED

import pandas as pd

# Load the data from the two CSV files
df_cardholders = pd.read_csv("UK_cardholder_Dom_HH.csv")
df_spend = pd.read_csv("UK_HH_Dom_Spending.csv")

# Merge the two DataFrames on 'time_period_value'
df_merged = pd.merge(df_cardholders, df_spend, on="time_period_value", how="inner")

# Extract the 2019Q1 values
base_row = df_merged[df_merged["time_period_value"] == "2019Q1"]
if not base_row.empty:
    base_cardholders = base_row["Dom_HH_All_cardholders"].values[0]
   
    # Calculate adjusted quarterly spend
    df_merged["adjusted_Dom_spend"] = (
        base_cardholders / df_merged["Dom_HH_All_cardholders"]
    ) * df_merged["Dom_spend_HH"]

    # Save the result to a new CSV file
    df_merged.to_csv("Adjusted_Dom_HH_Spend.csv", index=False)
    print("Adjusted quarterly spend saved to Adjusted_Dom_HH_Spend.csv")
else:
    print("2019Q1 base data not found in the dataset.")
print(df_merged)


In [ ]:
# Total UK Cardholder Household Abroad Quarterly

# Summarise the data by UK Cardholder Abroad Household Spending Total Quarterly

UK_spending_HH_Intl_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
and mcc = 'All'
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_HH_Intl_All = bq.read_bq_table_sql(client, UK_spending_HH_Intl_All)
df_by_HH_Intl_All = df_by_HH_Intl_All.rename(columns={'total_spend': 'Intl_spend_HH'})
df_by_HH_Intl_All.head()


# Save the DataFrame to a CSV file
csv_filename = "UK_HH_Intl_Spending.csv"
df_by_HH_Intl_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Formula : Adjusted Quarterly Spend=( Intl_HH_All_cardholders (2019Q1) / Intl_HH_All_cardholders (quarter) ) × Intl_spend_All (quarter)

# UK Household Abroad Total Adjusted base value 2019Q1  ---------------- MODIFIED

import pandas as pd

# Load the data from the two CSV files
df_cardholders = pd.read_csv("UK_cardholder_Intl_HH.csv")
df_spend = pd.read_csv("UK_HH_Intl_Spending.csv")

# Merge the two DataFrames on 'time_period_value'
df_merged = pd.merge(df_cardholders, df_spend, on="time_period_value", how="inner")

# Extract the 2019Q1 values
base_row = df_merged[df_merged["time_period_value"] == "2019Q1"]
if not base_row.empty:
    base_cardholders = base_row["Intl_HH_All_cardholders"].values[0]
   
    # Calculate adjusted quarterly spend
    df_merged["adjusted_Intl_spend"] = (
        base_cardholders / df_merged["Intl_HH_All_cardholders"]
    ) * df_merged["Intl_spend_HH"]

    # Save the result to a new CSV file
    df_merged.to_csv("Adjusted_Intl_HH_Spend.csv", index=False)
    print("Adjusted quarterly spend saved to Adjusted_Intl_HH_Spend.csv")
else:
    print("2019Q1 base data not found in the dataset.")
print(df_merged)


In [ ]:
# Summarise the data by UK Household Cardholder Domestic Online Spending All Quarterly --------------- Cardholders' Number Total Quarterly 

UK_cardholder_HH_Dom_Online = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM' 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_HH_Dom_Online = bq.read_bq_table_sql(client, UK_cardholder_HH_Dom_Online)
df_by_HH_Dom_Online.head()

# Caculate UK Domestic Online Total Spending Quarterly

# Assuming df_by_HH_Dom_Online is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_HH_Dom_Online is not None and has the expected columns
if df_by_HH_Dom_Online is not None and 'time_period_value' in df_by_HH_Dom_Online.columns and 'spend' in df_by_HH_Dom_Online.columns:
    # Group by quarter and sum the spend
    UK_cardholder_HH_Dom_Online = df_by_HH_Dom_Online.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_cardholder_HH_Dom_Online = UK_cardholder_HH_Dom_Online.rename(columns={'cardholders': 'Dom_HH_Online_cardholders'})
    print(UK_cardholder_HH_Dom_Online)
else:
    print("DataFrame is empty or missing required columns.")
    
# Save the result to a CSV file
csv_filename = "UK_cardholder_HH_Dom_Online.csv"
UK_cardholder_HH_Dom_Online.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")




In [ ]:
# Total UK Cardholder Household Domestic Online Quarterly

# Summarise the data by UK Cardholder Domestic Household Online Spending Total Quarterly

UK_spending_HH_Dom_Online = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
and mcc = 'All'  
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and cardholder_origin_country = 'All' 
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and destination_country = 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_HH_Dom_Online = bq.read_bq_table_sql(client, UK_spending_HH_Dom_Online)
df_by_HH_Dom_Online = df_by_HH_Dom_Online.rename(columns={'total_spend': 'Dom_Online_HH'})
df_by_HH_Dom_Online.head()


# Save the DataFrame to a CSV file
csv_filename = "UK_spending_HH_Dom_Online.csv"
df_by_HH_Dom_Online.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# UK Domestic Online Total Adjusted base value 2019Q1


# Formula : Adjusted Quarterly Spend=( Dom_HH_Online_cardholders (2019Q1) / Dom_HH_Online_cardholders (quarter) ) × Dom_spend_Online (quarter)

# UK Household Domestic Online Total Adjusted base value 2019Q1  ---------------- MODIFIED

import pandas as pd

# Load the data from the two CSV files
df_cardholders = pd.read_csv("UK_cardholder_HH_Dom_Online.csv")
df_spend = pd.read_csv("UK_spending_HH_Dom_Online.csv")

# Merge the two DataFrames on 'time_period_value'
df_merged = pd.merge(df_cardholders, df_spend, on="time_period_value", how="inner")

# Extract the 2019Q1 values
base_row = df_merged[df_merged["time_period_value"] == "2019Q1"]
if not base_row.empty:
    base_cardholders = base_row["Dom_HH_Online_cardholders"].values[0]
   
    # Calculate adjusted quarterly spend
    df_merged["adjusted_Dom_Online_spend"] = (
        base_cardholders / df_merged["Dom_HH_Online_cardholders"]
    ) * df_merged["Dom_Online_HH"]

    # Save the result to a new CSV file
    df_merged.to_csv("Adjusted_Dom_HH_Online_Spend.csv", index=False)
    print("Adjusted quarterly spend saved to Adjusted_Dom_HH_Online_Spend.csv")
else:
    print("2019Q1 base data not found in the dataset.")
print(df_merged)



In [ ]:
# Summarise the data by UK Household Cardholder Abroad Online Spending All Quarterly --------------- Cardholders' Number Total Quarterly 

UK_cardholder_HH_Intl_Online = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_HH_Intl_Online = bq.read_bq_table_sql(client, UK_cardholder_HH_Intl_Online)
df_by_HH_Intl_Online.head()

# Caculate UK Abroad Online Total Spending Quarterly

# Assuming df_by_HH_Intl_Online is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_HH_Intl_Online is not None and has the expected columns
if df_by_HH_Intl_Online is not None and 'time_period_value' in df_by_HH_Intl_Online.columns and 'spend' in df_by_HH_Intl_Online.columns:
    # Group by quarter and sum the spend
    UK_cardholder_HH_Intl_Online = df_by_HH_Intl_Online.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_cardholder_HH_Intl_Online = UK_cardholder_HH_Intl_Online.rename(columns={'cardholders': 'Intl_HH_Online_cardholders'})
    print(UK_cardholder_HH_Intl_Online)
else:
    print("DataFrame is empty or missing required columns.")
    
# Save the result to a CSV file
csv_filename = "UK_cardholder_HH_Intl_Online.csv"
UK_cardholder_HH_Intl_Online.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")




In [ ]:
# Total UK Cardholder Household Abroad Online Quarterly

# Summarise the data by UK Cardholder Abroad Household Online Spending Total Quarterly

UK_spending_HH_Intl_Online = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
and mcc = 'All'  
and mcg != 'All'
and mcg != 'BUSINESS TO BUSINESS' 
and cardholder_origin_country = 'All' 
and merchant_channel = 'Online'
and cardholder_origin = 'UNITED KINGDOM'
and destination_country != 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_HH_Intl_Online = bq.read_bq_table_sql(client, UK_spending_HH_Intl_Online)
df_by_HH_Intl_Online = df_by_HH_Intl_Online.rename(columns={'total_spend': 'Intl_Online_HH'})
df_by_HH_Intl_Online.head()


# Save the DataFrame to a CSV file
csv_filename = "UK_spending_HH_Intl_Online.csv"
df_by_HH_Intl_Online.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")


In [ ]:
# UK Abroad Online Total Adjusted base value 2019Q1


# Formula : Adjusted Quarterly Spend=( Intl_HH_Online_cardholders (2019Q1) / Intl_HH_Online_cardholders (quarter) ) × Intl_spend_Online (quarter)

# UK Household Abroad Online Total Adjusted base value 2019Q1  ---------------- MODIFIED

import pandas as pd

# Load the data from the two CSV files
df_cardholders = pd.read_csv("UK_cardholder_HH_Intl_Online.csv")
df_spend = pd.read_csv("UK_spending_HH_Intl_Online.csv")

# Merge the two DataFrames on 'time_period_value'
df_merged = pd.merge(df_cardholders, df_spend, on="time_period_value", how="inner")

# Extract the 2019Q1 values
base_row = df_merged[df_merged["time_period_value"] == "2019Q1"]
if not base_row.empty:
    base_cardholders = base_row["Intl_HH_Online_cardholders"].values[0]
   
    # Calculate adjusted quarterly spend
    df_merged["adjusted_Intl_Online_spend"] = (
        base_cardholders / df_merged["Intl_HH_Online_cardholders"]
    ) * df_merged["Intl_Online_HH"]

    # Save the result to a new CSV file
    df_merged.to_csv("Adjusted_Intl_HH_Online_Spend.csv", index=False)
    print("Adjusted quarterly spend saved to Adjusted_Intl_HH_Online_Spend.csv")
else:
    print("2019Q1 base data not found in the dataset.")
print(df_merged)


In [ ]:
# UK Cardholder Domestic Online Ratio with Adjusted Value Quarterly

import pandas as pd

# Load the datasets
online_spend = pd.read_csv("Adjusted_Dom_HH_Online_Spend.csv")
total_spend = pd.read_csv("Adjusted_Dom_HH_Spend.csv")

# Merge the datasets on the time period column
merged = pd.merge(online_spend, total_spend, on="time_period_value", suffixes=('_online', '_total'))

# Calculate the Domestic Online Ratio with Adjusted Value
merged["UK_Cardholder_Household_Domestic_Online_Ratio_Adjusted"] = (merged["adjusted_Dom_Online_spend"] / merged["adjusted_Dom_spend"]) * 100

# Save the result to a new CSV file
merged.to_csv("UK_Cardholder_Household_Domestic_Online_Ratio_Adjusted.csv", index=False)

# Display the resulting DataFrame
merged.head()




In [ ]:
# UK Cardholder Abroad Online Ratio with Adjusted Value Quarterly

import pandas as pd

# Load the datasets
online_spend = pd.read_csv("Adjusted_Intl_HH_Online_Spend.csv")
total_spend = pd.read_csv("Adjusted_Intl_HH_Spend.csv")

# Merge the datasets on the time period column
merged = pd.merge(online_spend, total_spend, on="time_period_value", suffixes=('_online', '_total'))

# Calculate the Domestic Online Ratio with Adjusted Value
merged["UK_Cardholder_Household_Intl_Online_Ratio_Adjusted"] = (merged["adjusted_Intl_Online_spend"] / merged["adjusted_Intl_spend"]) * 100

# Save the result to a new CSV file
merged.to_csv("UK_Cardholder_Household_Intl_Online_Ratio_Adjusted.csv", index=False)

# Display the resulting DataFrame
merged.head()




In [ ]:
# A line chart for UK_Cardholder_Household_Domestic_Online_Ratio_Adjusted.csv UK_Cardholder_Household_Intl_Online_Ratio_Adjusted.csv

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
intl_file = 'UK_Cardholder_Household_Intl_Online_Ratio_Adjusted.csv'
domestic_file = 'UK_Cardholder_Household_Domestic_Online_Ratio_Adjusted.csv'

# Read the data
intl_df = pd.read_csv(intl_file)
domestic_df = pd.read_csv(domestic_file)

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(intl_df['time_period_value'], intl_df['UK_Cardholder_Household_Intl_Online_Ratio_Adjusted'], label='International Adjusted Online Ratio', marker='s')
plt.plot(domestic_df['time_period_value'], domestic_df['UK_Cardholder_Household_Domestic_Online_Ratio_Adjusted'], label='Domestic Adjusted Online Ratio', marker='o')

# Formatting the plot
plt.title('UK Cardholder Household Online Ratio (Adjusted)')
plt.xlabel('Time Period')
plt.ylabel('Ratio (%)')
plt.xticks(rotation=45)
plt.legend(loc='center right')  # Legend placed at the middle right
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig('UK_Cardholder_Online_Ratio_Adjusted_Comparison.png')
print("Line chart saved as 'UK_Cardholder_Online_Ratio_Adjusted_Comparison.png'")





In [ ]:
# Summarise the data by UK Cardholder Domestic Face to Face Spending All Quarterly --------- Cardholders' Number Total Quarterly --------- Not Use

UK_spending_by_Dom_F2F = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg = 'All' 
and merchant_channel = 'Face to Face' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country = 'UNITED KINGDOM' 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_Dom_F2F = bq.read_bq_table_sql(client, UK_spending_by_Dom_F2F)
df_by_Dom_F2F.head()

# Caculate UK Domestic Face to Face Total Spending Quarterly

# Assuming df_by_Dom_F2F is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_Dom_F2F is not None and has the expected columns
if df_by_Dom_F2F is not None and 'time_period_value' in df_by_Dom_F2F.columns and 'spend' in df_by_Dom_F2F.columns:
    # Group by quarter and sum the spend
    UK_spending_by_Dom_F2F = df_by_Dom_F2F.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_spending_by_Dom_F2F = UK_spending_by_Dom_F2F.rename(columns={'cardholders': 'Dom_spend_F2F_cardholders'})
    print(UK_spending_by_Dom_F2F)
else:
    print("DataFrame is empty or missing required columns.")
    
# Save the result to a CSV file
csv_filename = "UK_spending_by_Dom_F2F.csv"
UK_spending_by_Dom_F2F.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")